# Training and evaluation of models

This notebook brings everything togehter to let us train and evaluate models in a simple manner. Due to no local GPU resources, this notebook contains code for google colab, so it **must be executed in google colab**.

<a target="_blank" href="https://colab.research.google.com/github/Moikii/SHEL">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

After downloading an existing dataset, or generating a new one (which takes more time for a large image size), can we train a model with some custom parameters. Evaluation follows next, where different models performance can be compared.

In [1]:
!rm -rf sample_data
!git clone https://github.com/Moikii/SHEL.git

Cloning into 'SHEL'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 81 (delta 33), reused 58 (delta 15), pack-reused 0
Receiving objects: 100% (81/81), 12.46 MiB | 7.49 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [17]:
%%capture
!pip install ultralytics

import sys
import os
from IPython.utils import io
from ultralytics import YOLO
py_file_location = "/content/SHEL/src"
sys.path.append(os.path.abspath(py_file_location))
import dataset_generation_functions as dgf
import photo_preparation_functions as ppf

In [18]:
new_dataset = True

# only when creating a new dataset
number_of_images = 100
max_number_of_cards_per_image = 4
min_size = 0.2
max_size = 0.7
max_rotation = 360
overlapping = True
seed = 42

# model training parameters
epochs = 2

In [21]:
if new_dataset:
  !. /content/SHEL/src/drive_data_download '1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5' 'backgrounds'
  !. /content/SHEL/src/drive_data_download '1o0f3q5zJzfRXs9vSMb1uR9KcUfWtFItZ' 'photos'
  DATASET_DIR = dgf.generate_dataset('/content/backgrounds', '/content/photos', '/content/dataset', number_of_images,
                                     max_number_of_cards_per_image, min_size, max_size, max_rotation, overlapping, seed)
  !rm -rf photos photos_processed backgrounds
else:
  !. /content/SHEL/src/drive_data_download '11cMzPUd_ETyOzzb0iJMtONWP10Nu7Zwc' 'dataset'
  print('Downloading dataset from external source...')
  DATASET_DIR = '/content/dataset'

--2023-10-23 22:10:22--  https://docs.google.com/uc?export=download&confirm=t&id=1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5
Resolving docs.google.com (docs.google.com)... 74.125.200.101, 74.125.200.102, 74.125.200.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-3k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7keu0os5op04b91igbt69fb52p8r3241/1698099000000/04921794579774064288/*/1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5?e=download&uuid=53cf616c-6d07-4168-a160-78a2d01b7d13 [following]
--2023-10-23 22:10:23--  https://doc-08-3k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7keu0os5op04b91igbt69fb52p8r3241/1698099000000/04921794579774064288/*/1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5?e=download&uuid=53cf616c-6d07-4168-a160-78a2d01b7d13
Resolving doc-08-3k-docs.googleusercontent.com (doc-08-3k-docs.googleusercontent.com)... 142.251.10.132, 

100%|██████████| 1000/1000 [01:33<00:00, 10.67it/s]

Dataset generated and saved at: /content/dataset


In [ ]:
model = YOLO('yolov8n.pt')
results = model.train(data = DATASET_DIR + '/data.yaml', epochs = epochs)


In [ ]:
# # Load a model
# model = YOLO('/home/moiki/Documents/Files/studies/4_Semester/ADL/ADL_W23/models/large_dataset_best_1.pt')

# # Validate the model
# metrics = model.val()  # no arguments needed, dataset and settings remembered
# metrics.box.map    # map50-95
# metrics.box.map50  # map50
# metrics.box.map75  # map75
# metrics.box.maps   # a list contains map50-95 of each category

In [20]:
!rm -rf runs
!rm -rf dataset